# *lsforce* example script: 2016 Lamplugh Glacier rock avalanche
---

This example script <span style="color:red">blah blah</span>

**References**

<span style="color:red">Blah blah</span>

### Import necessary modules

In [ ]:
import os

from obspy import UTCDateTime, read
from obspy.clients.fdsn import Client
from obspy.geodetics import gps2dist_azimuth

from lsforce import LSData, LSForce, LSTrajectory

# Ignore benign Matplotlib backend warning due to fig.show()
import warnings
warnings.filterwarnings(action='ignore', message='Matplotlib is currently using module')

### Define some constants, set up folder structure

In [ ]:
RUN_NAME = 'lamplugh'  # Nickname for this run

PERIOD_RANGE = (10, 200)  # [s] Bandpass filter corners

LS_LAT, LS_LON = (58.7371, -136.8943)  # Guesstimate from Google Earth + lamplugh.tif
ORIGIN_TIME = UTCDateTime(2016, 6, 28, 16, 21, 3)

STARTTIME = ORIGIN_TIME - 150
ENDTIME = UTCDateTime(2016, 6, 28, 16, 26, 6) + 150

# Set up folder structure
main_folder = os.path.join(os.getcwd(), RUN_NAME)
if not os.path.exists(main_folder):
    os.mkdir(main_folder)

### Gather inversion waveforms

In [ ]:
data_filename = os.path.join(main_folder, 'data.pkl')

# Download data if it doesn't exist as a file
if not os.path.exists(data_filename):

    client = Client('IRIS')
    waveform_kwargs = dict(
        location='', starttime=STARTTIME, endtime=ENDTIME, attach_response=True
    )

    # Gather waveforms
    NETWORKS = ('AK', 'AT', 'CN', 'TA')
    STATIONS = ('SKAG', 'BESE', 'P30M', 'JIS', 'PNL', 'YKU2', 'PLBC', 'SIT')
    st = client.get_waveforms(
        network=','.join(NETWORKS),
        station=','.join(STATIONS),
        channel='BH?,HH?',
        **waveform_kwargs,
    )

    # Grab coordinates
    inv = client.get_stations(
        network=','.join(NETWORKS),
        starttime=STARTTIME,
        endtime=ENDTIME,
        level='channel',
    )

    # Assign coordinates to Traces
    for tr in st:
        coords = inv.get_coordinates(tr.id, datetime=STARTTIME)
        tr.stats.latitude = coords['latitude']
        tr.stats.longitude = coords['longitude']

    st.write(data_filename, format='PICKLE')

# Use file if it exists, for speed
else:
    st = read(data_filename, format='PICKLE')

# Create LSData object
data = LSData(st, source_lat=LS_LAT, source_lon=LS_LON)

# Remove some noisy horizontals
for station in 'BESE', 'PNL', 'YKU2':
    for tr in data.st_proc.select(station=station, component='[RT]'):
        data.st_proc.remove(tr)

# Radial component of SKAG has poor fit...
for tr in data.st_proc.select(station='SKAG', component='R'):
    data.st_proc.remove(tr)

# Create plots
data.plot_stations(label_stations=True, gshhs_scale='h');
data.plot_data(period_range=PERIOD_RANGE);

### Gather reference waveform

In [ ]:
SEISMIC_VELO = 2  # [km/s] For travel time removal

client = Client('IRIS')

# Gather
st_hf = client.get_waveforms(
    network='CN',
    station='PLBC',
    location='',
    channel='HHZ',
    starttime=STARTTIME,
    endtime=ENDTIME,
    attach_response=True,
)

# Processing
st_hf.remove_response()
st_hf.detrend('demean')
st_hf.taper(max_percentage=0.05)
st_hf.filter('bandpass', freqmin=0.5, freqmax=5)

# Add "distance" to tr.stats
ref_inv = client.get_stations(
    network=st_hf[0].stats.network,
    station=st_hf[0].stats.station,
    starttime=STARTTIME,
    endtime=ENDTIME,
    level='channel',
)
for tr in st_hf:
    coords = ref_inv.get_coordinates(tr.id, datetime=STARTTIME)
    tr.stats.latitude = coords['latitude']
    tr.stats.longitude = coords['longitude']
    dist = gps2dist_azimuth(LS_LAT, LS_LON, tr.stats.latitude, tr.stats.longitude)[0]
    tr.stats.distance = dist / 1000  # [km]

# Approximate correction for travel time
hf_shift = st_hf[0].stats.distance / SEISMIC_VELO

### Setup

In [ ]:
force = LSForce(data=data, data_sampling_rate=1, main_folder=main_folder)

force.setup(
    period_range=PERIOD_RANGE,
    zerophase=True,
    syngine_model='iasp91_2s',
    weights='prenoise',
    noise_window_dur=150,
)

### Invert

In [ ]:
force.invert(
    zero_time=132,
    impose_zero_start=True,
    add_to_zero=True,
    jackknife=True,
    num_iter=20,
    frac_delete=0.25,
    alpha=1.9e-16,
    zero_scaler=2,
    tikhonov_ratios=(0, 0, 1),
)

### Plot inversion

In [ ]:
XLIM = (-20, 100)  # [s] x-axis (time) limits for plots

force.plot_fits(xlim=(-50, 200), equal_scale=False);
force.plot_forces(highf_tr=st_hf[0], hfshift=hf_shift, jackshowall=True, xlim=XLIM);
force.plot_angle_magnitude(xlim=XLIM);

### Compute trajectory

In [ ]:
L = 7  # [km] Estimate of horizontal COM runout length
DURATION = 100  # [s] Duration to use for trajectory integration

trajectory = LSTrajectory(force, target_length=L, duration=DURATION, detrend_velocity=DURATION)

### Plot trajectory

In [ ]:
trajectory.plot_trajectory(plot_jackknife=True);  # Horizontal trajectory
trajectory.plot_trajectory(plot_jackknife=True, elevation_profile=True);  # Vertical trajectory